In [1]:
import os
import math
import random

In [7]:
#using the Miller-Rabin test to find out if a number is prime or not.
def is_prime(n):
    """
    Test if a number is prime using the Miller-Rabin test.
    """
    if n < 2:
        return False
    if n == 2 or n == 3:
        return True
    if n % 2 == 0:
        return False
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2
    for i in range(10):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for i in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def generate_prime(bit_length):
    """
    Generate a probable prime number of the given bit length using the Miller-Rabin test.
    """
    while True:
        p = int.from_bytes(os.urandom(bit_length // 8), byteorder='big')
        p |= (1 << bit_length - 1) | 1 << bit_length - 2
        if is_prime(p):
            return p

def generate_key_pair(bit_length):
    """
    Generate an RSA key pair with the given bit length.
    """
    p = generate_prime(bit_length//2)
    q = generate_prime(bit_length//2)
    n = p * q
    phi = (p - 1) * (q - 1)

    # Generate a random value for e that is relatively prime to phi
    while True:
        e = random.randint(2, phi - 1)
        if math.gcd(e, phi) == 1:
            break

    d = mod_inverse(e, phi)
    return (n, e), (n, d)

def gcd(a, b):
    """
    Compute the greatest common divisor of two numbers using the Euclidean algorithm.
    """
    while b != 0:
        a, b = b, a % b
    return a

def mod_inverse(a, m):
    """
    Compute the modular inverse of a modulo m using the extended Euclidean algorithm.
    """
    if gcd(a, m) != 1:
        return None
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m

    while v3 != 0:
        q = u3 // v3
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3

    return u1 % m

def encrypt(message, public_key):
    """
    Encrypt a message using an RSA public key.
    """
    n, e = public_key
    block_size = int(math.log2(n) // 8)
    ciphertext = []
    for i in range(0, len(message), block_size):
        block = message[i:i+block_size]
        padded_block = block.ljust(block_size, '\x00')
        padded_int = int.from_bytes(padded_block.encode('utf-8'), byteorder='big')
        encrypted_int = pow(padded_int, e, n)
        ciphertext.append(encrypted_int)
    return ciphertext

def decrypt(ciphertext, private_key):
    """
    Decrypt a message using an RSA private key.
    """
    n, d = private_key
    block_size = int(math.log2(n) // 8)
    decrypted_blocks = []

    for encrypted_int in ciphertext:
        decrypted_int = pow(encrypted_int, d, n)
        padded_block_bytes = decrypted_int.to_bytes(block_size, byteorder='big')
        # Strip padding to retrieve the original message block
        original_block = padded_block_bytes.rstrip(b'\x00')
        decrypted_blocks.append(original_block.decode('utf-8'))

    # Concatenate all the decrypted blocks to form the original message
    original_message = ''.join(decrypted_blocks)
    return original_message

def save_key_to_file(key, filename):
    """
    Save an RSA key to a file.
    """
    if len(key) == 2:  # tuple form
        n, k = key
        with open(filename, 'wb') as f:
            f.write(n.to_bytes((n.bit_length() + 7) // 8, byteorder='big'))
            f.write(k.to_bytes((k.bit_length() + 7) // 8, byteorder='big'))
    else:
        raise ValueError("Invalid RSA key format.")

In [3]:
# Generate a key pair with 1024-bit p and q
bit_length = 2048
public_key, private_key = generate_key_pair(bit_length)

# Save the public and private keys to files
save_key_to_file(public_key, 'public.pem')
save_key_to_file(private_key, 'private.pem')

# Encrypt a message using the public key and save it to a file
message = "The quick brown fox jumps over the lazy dog"
encrypted_text = encrypt(message, public_key)
with open('encrypted.txt', 'w') as f:
    for c in encrypted_text:
        f.write(str(c))
        f.write('\n')

In [8]:
decrypted_text = decrypt(encrypted_text, private_key)


In [4]:
public_key

(26849866536184532786821404868385286413898579440680963925238569389203177403105361194337630075600658888175506113524315700632434872711552635594659599755525165376155796530010837493228578163471540667415645029979450790016076456229996057005524350110577166891355244191697531559989904391918564403937149865973545600195688776253160964604556922527709303166751060016231453333365017558656385744898940414719277695095065073119186761952297309330007325161644946048399511843123360671574213972131684605383026316338497694679733790076999246483169745184973772263788985820205752241648218518310849100117283976908259055225522061057883987079661,
 2655880582009495903643959442238523068626527312287637557621953993895829049375440285165537069533958759420201428685065449291692219641754011323947288551373253813435691734757066536904596739243405625450194490435026667632913584953194367293862834815924159286294987888775958051317830483671521734912487452353324277075785203605183287879755843064605497915181152534736814006111726457335408351

In [5]:
private_key

(26849866536184532786821404868385286413898579440680963925238569389203177403105361194337630075600658888175506113524315700632434872711552635594659599755525165376155796530010837493228578163471540667415645029979450790016076456229996057005524350110577166891355244191697531559989904391918564403937149865973545600195688776253160964604556922527709303166751060016231453333365017558656385744898940414719277695095065073119186761952297309330007325161644946048399511843123360671574213972131684605383026316338497694679733790076999246483169745184973772263788985820205752241648218518310849100117283976908259055225522061057883987079661,
 2580181437895233939710458836736981649658802523352323715062296020584943198882423549059048025185194007794467124520166303730511595856846686759460132314995246991289902497296619750265841075607034945390091745189393883586382829549929343518539785936215301233103059978545357449532146020894824490565215679517187532512900002213322820840883162633445716001605466833223901722175972768196692302

In [6]:
encrypted_text

[12628154666787804475561056964001668008971832852740027580714263720055111147808872598117999367285726693007620169178649484935504830352191526259217723322447314717695518927283496064766325720266449536255002689128682343255133738341116770980146700730249156016906277694501167975180578336210615032829489890356367759023984698585564012598948292951130647162152470286573737875083532108502117990482519211769547420729782089059639776640439050254660070332488866942188791369032267741072189994887582977925892426497886642218306028245420720561083995723892962687334718008332878093179078099910984796621067403449348061831874908852196935080689]

In [9]:
decrypted_text

'The quick brown fox jumps over the lazy dog'